# One-Class SVM for Traffic Anomaly Detection

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing as pp
from sklearn.svm import OneClassSVM
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
import matplotlib.pyplot as plt

## Load Data

In [2]:
#data = pd.read_csv('../Datasets/kitsune_dataset/SSDP_Flood/SSDP_Flood_dataset.csv', index_col=0, dtype=np.float32)
#labels = pd.read_csv('../Datasets/kitsune_dataset/SSDP_Flood/SSDP_Flood_labels.csv', index_col=0, dtype=np.int32)
data = pd.read_csv('../Datasets/kitsune_dataset/Active_Wiretap/Active_Wiretap_dataset.csv', index_col=0, dtype=np.float32)
labels = pd.read_csv('../Datasets/kitsune_dataset/Active_Wiretap/Active_Wiretap_labels.csv', index_col=0, dtype=np.int32)

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 2278688 entries, 1.9973247051239014 to 332.7803649902344
Columns: 114 entries, 2.090000000000000000e+02 to 0.000000000000000000e+00.54
dtypes: float32(114)
memory usage: 1008.3 MB


In [4]:
labels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2278689 entries, 1 to 2278689
Data columns (total 1 columns):
 #   Column  Dtype
---  ------  -----
 0   x       int32
dtypes: int32(1)
memory usage: 26.1 MB


## Scale Data

In [5]:
scaler = pp.MinMaxScaler()
data = scaler.fit_transform(data)
data = pd.DataFrame(data)
data.describe()

,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,113
count,2.278688e+06,2.278688e+06,2.278688e+06,2.278688e+06,2.278688e+06,2.278688e+06,2.278688e+06,2.278688e+06,2.278688e+06,2.278688e+06,...,2.278688e+06,2.278688e+06,2.278688e+06,2.278688e+06,2.278688e+06,2.278688e+06,2.278688e+06,2.278688e+06,2.278688e+06,2.278688e+06
mean,8.222400e-01,3.050174e-01,5.430585e-01,8.226944e-01,3.036735e-01,6.130465e-01,8.231573e-01,3.058952e-01,6.388294e-01,9.045548e-01,...,3.271092e-01,5.798556e-01,4.435584e-01,6.684283e-01,8.808641e-01,5.449103e-01,9.242014e-01,3.270650e-01,5.824689e-01,4.416649e-01
std,1.945486e-01,8.602510e-02,2.220364e-01,1.944936e-01,8.325344e-02,2.463958e-01,1.944886e-01,8.199255e-02,2.640495e-01,2.136368e-01,...,9.835514e-02,4.594623e-03,8.572696e-03,2.620880e-01,2.113756e-01,1.556235e-01,5.621867e-02,9.787289e-02,4.652583e-03,1.008293e-02
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.506727e-01,2.760243e-01,3.670852e-01,8.512515e-01,2.729489e-01,4.148379e-01,8.506180e-01,2.727349e-01,4.398870e-01,9.339065e-01,...,3.675718e-01,5.799512e-01,4.438865e-01,4.752771e-01,9.380996e-01,6.083499e-01,9.373477e-01,3.700897e-01,5.826010e-01,4.425227e-01
50%,8.692419e-01,3.233634e-01,5.141586e-01,8.723988e-01,3.296277e-01,5.494377e-01,8.765783e-01,3.420107e-01,5.686493e-01,9.657908e-01,...,3.709228e-01,5.799512e-01,4.438865e-01,7.865087e-01,9.421977e-01,6.091749e-01,9.414440e-01,3.710942e-01,5.826010e-01,4.425227e-01
75%,8.807910e-01,3.565571e-01,7.468726e-01,8.808829e-01,3.530799e-01,8.264682e-01,8.805224e-01,3.537428e-01,8.531925e-01,9.667862e-01,...,3.734404e-01,5.799512e-01,4.438865e-01,8.771517e-01,9.425868e-01,6.100653e-01,9.418011e-01,3.721799e-01,5.826010e-01,4.425227e-01
max,1.000000e+00,1.000000e+00,9.999999e-01,1.000000e+00,1.000000e+00,1.000000e+00,9.999999e-01,1.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,9.999999e-01,1.000000e+00,1.000000e+00,9.999999e-01,1.000000e+00,1.000000e+00,1.000000e+00


## Train-Test Split

In [6]:
# drop the last row
labels=labels[:-1]
labels.tail(5)
# train-test split
X_train, X_test, y_train, y_test = train_test_split(data, labels, shuffle=False, test_size=0.3)

In [7]:
# anomaly counts
print(f'Number anomalies in train data: {y_train.sum()}')
print(f'Number anomalies in test data: {y_test.sum()}')

Number anomalies in train data: x    279997
dtype: int64
Number anomalies in test data: x    643218
dtype: int64


## Create Model

In [ ]:
model = OneClassSVM(kernel='rbf')
model.fit(X_train)

## Evaluate

In [ ]:
predictions = model.predict(X_test)

In [ ]:
# convert to 0-1
predictions = [0 if x>0 else 1 for x in predictions] 

In [ ]:
# average precision score train
aps_test = average_precision_score(y_test, predictions)
print(f'Average precision score for test: {aps_test:.4f}')

In [ ]:
# precision-recall curve for test results
precision, recall, thresholds = precision_recall_curve(y_test, predictions)
plt.title(f'Средняя точность: {aps_test*100:.1f}%')
plt.xlabel('Чувствительность')
plt.xlim([0, 1.1])
plt.ylim([0, 1.1])
plt.ylabel('Точность')
#plt.ylim([0, 1.1])
#plt.xlim([0, 1.0])
plt.grid()
plt.plot(recall, precision, color='k')
plt.savefig('OSVM_Active_Wiretap_AP.png')
plt.show()

In [ ]:
# receiver operational characteristic
fpr, tpr, thresholds = roc_curve(y_test, predictions)
auROC = auc(fpr, tpr)
plt.figure()
plt.title(f'Площадь под кривой={auROC:.2f}')
plt.plot(fpr, tpr, color='k', lw=2)
plt.xlabel('FPR')
plt.ylabel('TPR')
plt.plot([0,1], [0,1], color='k', lw=2, linestyle='--')
plt.grid()
plt.savefig('OSVM_Active_Wiretap_AUC.png')
plt.show()